In [1]:
# Imports
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.engine import result
from sqlalchemy import inspect
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
from dotenv import dotenv_values


In [2]:
# Grab variables from .env and create engine for accessing the database

config = dotenv_values(".env")

user = config['DB_USER']
password = config['DB_PASSWORD']
host = config['HOST']
port = config['PORT']
database = config['DATABASE']
remove_ids = config['REMOVE_IDS']
 

engine = create_engine(f"oracle+cx_oracle://{user}:{password}@{host}:{port}/?service_name={database}&encoding=UTF-8&nencoding=UTF-8", echo=False)

today_date = datetime.today().strftime('%m-%d-%Y')

# Visualization/analysis data

In the following we'll extract the visualizations and computations used per analysis using Oracle's `json_table` function, and save the output as a csv.

In [3]:
# Warning - this could take 25 min to run
query = f"""
  select eda.analysis_id
    , to_char(eda.creation_time, 'DD-MM-yyyy') as creation_day
    , analysis_descriptor_table.*
    , eda.analysis_descriptor as analysis_descriptor_json
  from edausermb.analysis eda,
    json_table(eda.analysis_descriptor, '$'
      columns (
        nested path '$.computations[*]' columns (
            computationType varchar2(50) path '$.descriptor.type',
            nested path '$.visualizations[*]' columns (
                visualizationType varchar2(50) path '$.descriptor.type'
            )
        )
      )
    ) analysis_descriptor_table
  where eda.user_id not in ({remove_ids})
  order by creation_day, eda.analysis_id
"""

sql = text(query)
with engine.connect() as conn:
    result = conn.execute(sql)
    df = pd.DataFrame(result)

df.head()

,analysis_id,creation_day,computationtype,visualizationtype
0,873cdtI,01-01-2024,betadiv,scatterplot
1,B5SbgsA,01-01-2024,None,None
2,neV8Byf,01-01-2024,betadiv,scatterplot
3,neV8Byf,01-01-2024,alphadiv,boxplot
4,neV8Byf,01-01-2024,betadiv,scatterplot


In [4]:
# Write to csv
df.to_csv(f"analysis_descriptor_table_{today_date}.csv", index=False)